In [5]:
from langchain_core.documents import Document

In [8]:
doc = Document(
    page_content="this is the main content i am using to create rag",
    metadata={
        "source": "exmaple.txt",
        "pages":1,
        "author":"Varun Bharadwaj S",
        "data_created":"2026-11-02"
    }
)
doc

Document(metadata={'source': 'exmaple.txt', 'pages': 1, 'author': 'Varun Bharadwaj S', 'data_created': '2026-11-02'}, page_content='this is the main content i am using to create rag')

In [11]:
import os
os.makedirs("../data/text_files",exist_ok=True)

In [ ]:
sample_text={
    "../data/text_files/python_intro.txt":"""Python is a high-level, interpreted, general-purpose programming language. 
    Created by Guido van Rossum and first released in 1991, Python emphasizes code readability with its notable use of significant indentation. 
    It features a dynamic type system and automatic memory management, supporting multiple programming paradigms, including procedural, object-oriented, and functional programming.
    Python's extensive standard library and simple syntax make it a popular choice for web development, data science, artificial intelligence, and automation."""
}

for file_path,content in sample_text.items():
    with open(file_path,"w",encoding="utf-8") as f:
        f.write(content)
print("done creating file")

In [3]:
from langchain_community.document_loaders import TextLoader


loader = TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content="Python is a high-level, interpreted, general-purpose programming language. \nCreated by Guido van Rossum and first released in 1991, Python emphasizes code readability with its notable use of significant indentation. \nIt features a dynamic type system and automatic memory management, supporting multiple programming paradigms, including procedural, object-oriented, and functional programming.\nPython's extensive standard library and simple syntax make it a popular choice for web development, data science, artificial intelligence, and automation.")]


In [9]:
from langchain_community.document_loaders import DirectoryLoader

dir_loader=DirectoryLoader("../data/text_files"
,glob="**/*.txt",
loader_cls= TextLoader,
loader_kwargs={
    "encoding":"utf-8"
},
show_progress=False

)

documents= dir_loader.load()
documents

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content="Python is a high-level, interpreted, general-purpose programming language. \nCreated by Guido van Rossum and first released in 1991, Python emphasizes code readability with its notable use of significant indentation. \nIt features a dynamic type system and automatic memory management, supporting multiple programming paradigms, including procedural, object-oriented, and functional programming.\nPython's extensive standard library and simple syntax make it a popular choice for web development, data science, artificial intelligence, and automation.")]

In [6]:
from langchain_community.document_loaders import DirectoryLoader,PyMuPDFLoader, PyPDFLoader


dir_loader = DirectoryLoader("../data/pdf"
,glob="**/*.pdf",
loader_cls= PyMuPDFLoader,
show_progress=False

)

pdf_documents= dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'macOS Version 26.2 (Build 25C56) Quartz PDFContext', 'creator': '', 'creationdate': "D:20260211092504Z00'00'", 'source': '../data/pdf/Rag Application and its use cases.pdf', 'file_path': '../data/pdf/Rag Application and its use cases.pdf', 'total_pages': 4, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20260211092504Z00'00'", 'trapped': '', 'modDate': "D:20260211092504Z00'00'", 'creationDate': "D:20260211092504Z00'00'", 'page': 0}, page_content='Rag Application and its use cases \n \n1. What is RAG? \n• Full form: retrieval argument, generation \n• It is the retrieval of relevant data from the source according to \nuser requirement \n• Referred to information outside of its training data source \n \n2. What is similarity search? \n• A type of search which composers the similarity using queries \n• Pardon is a technique to convert data into chunks or break them \ndown into diFerent parts \n• Embedding is a t

In [7]:
type(pdf_documents[0])


langchain_core.documents.base.Document